<a href="https://colab.research.google.com/github/nhungnth406/DMproj_ChurnRate_Predict/blob/main/Classification_ChurnRate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TIỀN XỬ LÝ DỮ LIỆU

## Thư viện kèm theo


In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from pylab import rcParams
%matplotlib inline

In [2]:
from google.colab import files
uploaded = files.upload()

Saving TelcoCustomerChurn.csv to TelcoCustomerChurn.csv


In [3]:
data = pd.read_csv("/content/TelcoCustomerChurn.csv")

## Xóa những thuộc tính không ý nghĩa trong quá trình khai thác


In [4]:
data.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

Có 7043 đối tượng dữ liệu và 21 thuộc tính



Xóa cột customerID vì đây là dữ liệu không cần thiết để tìm ra tỉ lệ khách hàng hủy đăng ký dịch vụ

In [5]:
data.drop(['customerID'], axis=1, inplace = True)

In [6]:
# Tìm dữ liệu bị thiếu
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
Churn,0,0.0
TotalCharges,0,0.0
SeniorCitizen,0,0.0
Partner,0,0.0
Dependents,0,0.0
tenure,0,0.0
PhoneService,0,0.0
MultipleLines,0,0.0
InternetService,0,0.0
OnlineSecurity,0,0.0


## Thuộc tính Gender

**<font color='royalblue'> Thuộc tính Gender: Giới tính của khách hàng (Nam - Male, Nữ - Female)</font>**



Chuyển giá trị của cột Gender từ giá trị Male, Female về giá trị 0 và 1 để phù hợp với đầu vào của thuật toán

In [7]:
data['gender'].head()

0    Female
1      Male
2      Male
3      Male
4    Female
Name: gender, dtype: object

In [8]:
data['gender'] = data['gender'].map({'Male': 1, 'Female': 0})
data['gender'].value_counts()

1    3555
0    3488
Name: gender, dtype: int64

## Family (Partner + Dependents)

In [9]:
#Creating a new column for family. If a customer has dependant or Partner, I am considering it as family .
list_family = []
for rows in range(len(data['Partner'])):
    if ((data['Partner'][rows] == 'No') and (data['Dependents'][rows] == 'No')):
        list_family.append('No')
    else:
        list_family.append('Yes')
data['Family'] = list_family

In [10]:
# For Partner & Dependant , we created Family Column . Converting Family column to numeric value
data['Family'] = data['Family'].map( {'Yes': 1, 'No': 0} ).astype(int) #Map Categorical to Numerical Values

## Online Service (OnlineSecurity + OnlineBackup)

In [11]:
#Creating a new column for Online Services (Online Security & Online Backup) . If a customer has Online Security or Online Backup services
#then , I am considering it as "Yes" else "No"
list_online_services = []
for rows_os in range(len(data['OnlineSecurity'])):
    if ((data['OnlineSecurity'][rows_os] == 'No') and (data['OnlineBackup'][rows_os] == 'No')):
        list_online_services.append('No')
    else:
        list_online_services.append('Yes')
data['OnlineServices'] = list_online_services



## Streaming Service (StreamingTV + StreamingMovies)

In [12]:
#Creating a new column for Streaming Services (StreamingTV & StreamingMovies) . If a customer has StreamingTV or StreamingMovies
#then , I am considering it as "Yes" else "No"
list_streaming_services = []
for rows_stv in range(len(data['StreamingTV'])):
    if ((data['StreamingTV'][rows_stv] == 'No') and (data['StreamingMovies'][rows_stv] == 'No')):
        list_streaming_services.append('No')
    else:
        list_streaming_services.append('Yes')
data['StreamingServices'] = list_streaming_services


## Các thuộc tính khác

In [13]:
data['OnlineServices'] = data['OnlineServices'].map( {'Yes': 1, 'No': 0} ).astype(int) #Map Categorical to Numerical Values
data['StreamingServices'] = data['StreamingServices'].map({'Yes': 1, 'No': 0})
data['PaperlessBilling'] = data['PaperlessBilling'].map({'Yes': 1, 'No': 0})
data['PhoneService'] = data['PhoneService'].map({'Yes': 1, 'No': 0})


**<font color='royalblue'> Thuộc tính MonthlyCharges: Số tiền khách hàng trả hàng tháng thuê dịch vụ</font>**

In [14]:
data['MonthlyCharges'].head(12)

0      29.85
1      56.95
2      53.85
3      42.30
4      70.70
5      99.65
6      89.10
7      29.75
8     104.80
9      56.15
10     49.95
11     18.95
Name: MonthlyCharges, dtype: float64

**<font color='royalblue'> Thuộc tính TotalCharges: Tổng chi phí khách hàng đã chi trả</font>**


In [15]:
data['TotalCharges'].head()

0      29.85
1     1889.5
2     108.15
3    1840.75
4     151.65
Name: TotalCharges, dtype: object

In [16]:
## Có 11 hàng chứa " ", đây là 11 hàng bị thiếu dữ liệu
len(data[data['TotalCharges'] == " "])

11

In [17]:
## Xóa các hàng thiếu dữ liệu
data = data[data['TotalCharges'] != " "]

In [18]:
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])

**<font color='tomato'>Kết luận</font>**

    Hầu hết khách hàng ngừng sử dụng dịch vụ có tổng chi phí chi trả chưa đến 2000$ 
    Trong khoảng chi phí từ 2500$ đến 8000$, số khách hàng trung thành cao gấp đôi so với khách hàng ngừng sử dụng dịch vụ.

## Rời rạc hóa dữ liệu

<font color='royalblue'>**Rời rạc hóa dữ liệu**

In [19]:
data['quantile_MonthlyCharges'] = pd.qcut(data['MonthlyCharges'], q = 4)

In [20]:
data['quantile_MonthlyCharges'].value_counts()

(35.588, 70.35]     1761
(89.862, 118.75]    1758
(18.249, 35.588]    1758
(70.35, 89.862]     1755
Name: quantile_MonthlyCharges, dtype: int64

In [21]:
for data2 in [data]:
  data2['MonthlyCharges'] = data2['MonthlyCharges'].astype(int)
  data2.loc[(data['MonthlyCharges'] > 18.249) & (data2['MonthlyCharges'] <= 35.588), 'MonthlyCharges'] = 1;
  data2.loc[(data['MonthlyCharges'] > 35.588) & (data2['MonthlyCharges'] <= 70.35), 'MonthlyCharges'] = 2;
  data2.loc[(data['MonthlyCharges'] > 70.35) & (data2['MonthlyCharges'] <= 89.862), 'MonthlyCharges'] = 3;
  data2.loc[(data['MonthlyCharges'] > 89.862) & (data2['MonthlyCharges'] <= 118.75), 'MonthlyCharges'] = 4;

In [22]:
data['MonthlyCharges'].head(12)

0      1
1      2
2      2
3      2
4      2
5      4
6      3
7      1
8      4
9      2
10     2
11    18
Name: MonthlyCharges, dtype: int64

In [23]:
data.head(12)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Family,OnlineServices,StreamingServices,quantile_MonthlyCharges
0,0,0,Yes,No,1,0,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,1,Electronic check,1,29.85,No,1,1,0,"(18.249, 35.588]"
1,1,0,No,No,34,1,No,DSL,Yes,No,Yes,No,No,No,One year,0,Mailed check,2,1889.50,No,0,1,0,"(35.588, 70.35]"
2,1,0,No,No,2,1,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,1,Mailed check,2,108.15,Yes,0,1,0,"(35.588, 70.35]"
3,1,0,No,No,45,0,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,0,Bank transfer (automatic),2,1840.75,No,0,1,0,"(35.588, 70.35]"
4,0,0,No,No,2,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,2,151.65,Yes,0,0,0,"(70.35, 89.862]"
5,0,0,No,No,8,1,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,1,Electronic check,4,820.50,Yes,0,0,1,"(89.862, 118.75]"
6,1,0,No,Yes,22,1,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,1,Credit card (automatic),3,1949.40,No,1,1,1,"(70.35, 89.862]"
7,0,0,No,No,10,0,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,0,Mailed check,1,301.90,No,0,1,0,"(18.249, 35.588]"
8,0,0,Yes,No,28,1,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,1,Electronic check,4,3046.05,Yes,1,0,1,"(89.862, 118.75]"
9,1,0,No,Yes,62,1,No,DSL,Yes,Yes,No,No,No,No,One year,0,Bank transfer (automatic),2,3487.95,No,1,1,0,"(35.588, 70.35]"


In [24]:
data['quantile_TotalCharges'] = pd.qcut(data['TotalCharges'], q = 4)

In [25]:
data['quantile_TotalCharges'].value_counts().sort_values(ascending=True)

(3794.738, 8684.8]      1758
(1397.475, 3794.738]    1758
(401.45, 1397.475]      1758
(18.799, 401.45]        1758
Name: quantile_TotalCharges, dtype: int64

In [26]:
for data3 in [data]:
  data3['TotalCharges'] = data3['TotalCharges'].astype(int)
  data3.loc[(data['TotalCharges'] > 18.799) & (data3['TotalCharges'] <= 401.45), 'TotalCharges'] = 1;
  data3.loc[(data['TotalCharges'] > 401.45) & (data3['TotalCharges'] <= 1397.475), 'TotalCharges'] = 2;
  data3.loc[(data['TotalCharges'] > 1397.475) & (data3['TotalCharges'] <= 3794.738), 'TotalCharges'] = 3;
  data3.loc[(data['TotalCharges'] > 3794.738) & (data3['TotalCharges'] <= 8684.8), 'TotalCharges'] = 4;

In [27]:
data['TotalCharges']

0       1
1       3
2       1
3       3
4       1
       ..
7038    3
7039    4
7040    1
7041    1
7042    4
Name: TotalCharges, Length: 7032, dtype: int64

In [28]:
data['quantile_tenure'] = pd.qcut(data['tenure'], q = 4)

In [29]:
data['quantile_tenure'].value_counts()

(0.999, 9.0]    1843
(55.0, 72.0]    1755
(29.0, 55.0]    1719
(9.0, 29.0]     1715
Name: quantile_tenure, dtype: int64

In [30]:
for data4 in [data]:
  data4['tenure'] = data4['tenure'].astype(int)
  data4.loc[(data['tenure'] > 0.999) & (data4['tenure'] <= 9.0), 'tenure'] = 1;
  data4.loc[(data['tenure'] > 9.0) & (data4['tenure'] <= 29.0), 'tenure'] = 2;
  data4.loc[(data['tenure'] > 29.0) & (data4['tenure'] <= 55.0), 'tenure'] = 3;
  data4.loc[(data['tenure'] > 55.0) & (data4['tenure'] <= 72.0), 'tenure'] = 4;

In [31]:
data['tenure']

0       1
1       3
2       1
3       3
4       1
       ..
7038    2
7039    4
7040    2
7041    1
7042    4
Name: tenure, Length: 7032, dtype: int64

In [32]:
data.drop(['quantile_MonthlyCharges', 'quantile_TotalCharges', 'quantile_tenure'], axis=1, inplace = True)
data

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Family,OnlineServices,StreamingServices
0,0,0,Yes,No,1,0,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,1,Electronic check,1,1,No,1,1,0
1,1,0,No,No,3,1,No,DSL,Yes,No,Yes,No,No,No,One year,0,Mailed check,2,3,No,0,1,0
2,1,0,No,No,1,1,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,1,Mailed check,2,1,Yes,0,1,0
3,1,0,No,No,3,0,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,0,Bank transfer (automatic),2,3,No,0,1,0
4,0,0,No,No,1,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,2,1,Yes,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,Yes,Yes,2,1,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,1,Mailed check,3,3,No,1,1,1
7039,0,0,Yes,Yes,4,1,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,1,Credit card (automatic),4,4,No,1,1,1
7040,0,0,Yes,Yes,2,0,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,1,Electronic check,1,1,No,1,1,0
7041,1,1,Yes,No,1,1,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Mailed check,3,1,Yes,1,0,0


In [33]:
data.drop(['Partner', 'Dependents', 'OnlineSecurity', 'OnlineBackup', 'StreamingTV', 'StreamingMovies'], axis=1, inplace = True)

In [34]:
data.head(5)

,gender,SeniorCitizen,tenure,PhoneService,MultipleLines,InternetService,DeviceProtection,TechSupport,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Family,OnlineServices,StreamingServices
0,0,0,1,0,No phone service,DSL,No,No,Month-to-month,1,Electronic check,1,1,No,1,1,0
1,1,0,3,1,No,DSL,Yes,No,One year,0,Mailed check,2,3,No,0,1,0
2,1,0,1,1,No,DSL,No,No,Month-to-month,1,Mailed check,2,1,Yes,0,1,0
3,1,0,3,0,No phone service,DSL,Yes,Yes,One year,0,Bank transfer (automatic),2,3,No,0,1,0
4,0,0,1,1,No,Fiber optic,No,No,Month-to-month,1,Electronic check,2,1,Yes,0,0,0


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   gender             7032 non-null   int64 
 1   SeniorCitizen      7032 non-null   int64 
 2   tenure             7032 non-null   int64 
 3   PhoneService       7032 non-null   int64 
 4   MultipleLines      7032 non-null   object
 5   InternetService    7032 non-null   object
 6   DeviceProtection   7032 non-null   object
 7   TechSupport        7032 non-null   object
 8   Contract           7032 non-null   object
 9   PaperlessBilling   7032 non-null   int64 
 10  PaymentMethod      7032 non-null   object
 11  MonthlyCharges     7032 non-null   int64 
 12  TotalCharges       7032 non-null   int64 
 13  Churn              7032 non-null   object
 14  Family             7032 non-null   int64 
 15  OnlineServices     7032 non-null   int64 
 16  StreamingServices  7032 non-null   int64 


In [36]:
data.to_csv('TelcoCustomerChurn_Preprocessed.csv')

# THỰC NGHIỆM

## Nạp thư viện và chuẩn bị dữ liệu

In [37]:
%matplotlib inline 
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns 
from sklearn import tree 
import pandas as pd 
import pydotplus
from IPython.display import Image
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from datetime import datetime, timedelta
import time
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
#Đọc dữ liệu và xóa cột thừa
data = pd.read_csv('TelcoCustomerChurn_Preprocessed.csv')
data = data.drop(['Unnamed: 0'],axis=1)
data

In [ ]:
#Tách các thuộc tính và output thành 2 bảng lần lượt là: feature và label
feature = data.drop(['Churn'],axis=1)
label = data['Churn']

In [ ]:
#Chuyển đổi dữ liệu không phải số về dạng số bằng One-hot vector
feature_onehot = pd.get_dummies(feature, columns = feature.select_dtypes(exclude=['int64']).columns)
feature_onehot

In [ ]:
#Chia dữ liệu thành train và test theo tỉ lệ train:test = 8:2 
X_train,X_test,y_train,y_test = train_test_split(feature_onehot ,label,test_size=0.2,random_state=42)

## Decision Tree

In [ ]:
#Tiến hành train mô hình Decision tree bằng thuật toán CART 
Start_CART = time.time()

#Tạo cây quyết định với CART 
clfcart= tree.DecisionTreeClassifier(criterion ='gini', random_state=0)

#Train cây quyết định với CART 
clfcart_train = clfcart.fit(X_train,y_train)
End_CART = time.time()
tree_predict1 = clfcart_train.predict(X_test)
print('Training size= %d, Model accuracy= %.2f%%'% \
(X_train.shape[0], accuracy_score(y_test,tree_predict1)*100))
#Tính toán thời gian thuật toán chạy 
thoigian1 = timedelta(seconds=round(End_CART - Start_CART,4))
print('Thời gian chạy thuật toán là: ' ,thoigian1)


In [ ]:
#Dựng Confuse Matrix
cm = confusion_matrix(y_test, tree_predict1)
cm_matrix = pd.DataFrame(data=cm, columns=['Actual No:0','Actual Yes:1' ], 
                                 index=['Predict No:0','Predict Yes:1'])

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

In [ ]:
TP = cm[1,1]
TN = cm[0,0]
FP = cm[0,1]
FN = cm[1,0] 
print(TP,TN,FP,FN)

In [ ]:

print(classification_report(y_test, tree_predict1))

## Random Forest

In [ ]:
start_Random=time.time()
random_forest_ = RandomForestClassifier(100)
random_forest_.fit(X_train, y_train)
end_Random=time.time()
y_pred_rf = random_forest_.predict(X_test)
print('Training size= %d, Model accuracy= %.2f%%'% \
(X_train.shape[0], accuracy_score(y_test,y_pred_rf)*100))
timeofRF=timedelta(seconds=round(end_Random-start_Random,4))
print("Thời gian chạy thuật toán: ",timeofRF)

In [ ]:
cm_rf = confusion_matrix(y_test,y_pred_rf)

# %% confusion matrix visualization
import seaborn as sns
f, ax = plt.subplots(figsize = (5,5))
sns.heatmap(cm_rf, annot = True, linewidths = 0.5, color = "red", fmt = ".0f", ax=ax)
plt.xlabel("Predict")
plt.ylabel("Actual")
plt.title("Confusion Matrix of Random Forest")
plt.show()

In [ ]:
TP = cm_rf[1,1]
TN = cm_rf[0,0]
FP = cm_rf[0,1]
FN = cm_rf[1,0] 
print(TP,TN,FP,FN)

In [ ]:
print(classification_report(y_test, y_pred_rf))

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
start_Bayes = time.time()
# Khởi tạo thuật toán
gnb = GaussianNB()
# Training modelling
y_pred_bayes = gnb.fit(X_train,y_train).predict(X_test)
end_Bayes = time.time()
print('Training size= %d, Model accuracy= %.2f%%'% \
(X_train.shape[0], accuracy_score(y_test,y_pred_bayes)*100))
tgian= timedelta(seconds=round(end_Bayes-start_Bayes,4))
print("Thời gian chạy thuật toán:",tgian)

In [ ]:
cm_bayes = confusion_matrix(y_test,y_pred_bayes)

# %% confusion matrix visualization
import seaborn as sns
f, ax = plt.subplots(figsize = (5,5))
sns.heatmap(cm_bayes, annot = True, linewidths = 0.5, color = "red", fmt = ".0f", ax=ax)
plt.xlabel("Predict")
plt.ylabel("Actual")
plt.title("Confusion Matrix of Naive Bayes")
plt.show()

In [ ]:
TP = cm_bayes[1,1]
TN = cm_bayes[0,0]
FP = cm_bayes[0,1]
FN = cm_bayes[1,0] 
print(TP,TN,FP,FN)

In [ ]:
print(classification_report(y_test, y_pred_bayes))

# ĐÁNH GIÁ MÔ HÌNH

# Đo thời gian của 3 thuật toán

In [ ]:
plt.bar(x = ['CART', 'Random Forest','Naive Bayes' ], height = [End_CART - Start_CART, end_Random - start_Random,end_Bayes - start_Bayes], width = 0.4, color = '#646D78')
plt.xlabel('Tên thuật toán')
plt.ylabel('Thời gian chạy thuật toán')

# Đo độ chính xác của 3 thuật toán

In [ ]:
plt.bar(x = ['CART', 'Random Forest','Naive Bayes'], height = [accuracy_score(y_test,tree_predict1), accuracy_score(y_test,y_pred_rf),accuracy_score(y_test,y_pred_bayes),], width = 0.4, color = '#49CFAE')
plt.xlabel('Tên thuật toán')
plt.ylabel('Độ chính xác Accuracy')

# Đo F1 Score của 3 thuật toán

In [ ]:
plt.bar(x = ['CART','Random Forest','Naive Bayes'], height = [0.73, 0.75, 0.73], width = 0.4, color = '#FFCE54')
plt.xlabel('Tên thuật toán')
plt.ylabel('F1 Score')

# Dự đoán kết quả bằng Random Forest

In [ ]:
X_new=[[0,0,3,0,1,2,3,0,0,1,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0]]
y_new = random_forest_.predict(X_new)
print(y_new)